In [1]:
from main import df_raw
import pandas as pd
df = df_raw.df_accumulated()

In [3]:
base_index = 1000
store_index = base_index

df.size
df.shape[0]
df.describe()
df_raw.info

# Store_DF
- base    : new_store_df 
- columns : storages_columns : ['FN','OPD',"FD",'storage_name','adres','_date','sum_rub']

In [5]:
from decimal import *

store_infor_raw = df.copy()

store_info_index = store_infor_raw[
    (store_infor_raw['c0'].str.contains('ФН', na=False))
    &
    (store_infor_raw['c1'].str.contains('ФПД', na=False))] \
        .index.tolist()

store_df = store_infor_raw.iloc[list(map(lambda x: (x+1),store_info_index))]
new_store_df= store_df.reset_index(drop= True)

storages_columns = ['FN','OPD',"FD",'storage_name','adres','_date','sum_rub']
new_store_df.columns=storages_columns

new_store_df['_index_purch'] = new_store_df.index+base_index+1 # make inexes 1000+1

new_store_df['sum_rub'] = new_store_df['sum_rub'].apply(Decimal) # convert 'sum_rub' Decimal
new_store_df['_date'] = pd.to_datetime(new_store_df['_date'])

# new_store_df

,FN,OPD,FD,storage_name,adres,_date,sum_rub,_index_purch
0,8710000101342606,3980917342,3102,"ООО ""Агроторг""","129336 г. Москва, ул. Стартовая, д. 33А;",2018-01-25 18:26:00+00:00,557.68,1001
1,9289000100549090,986574045,5689,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МВМ""","Адрес расчета:Москва, ул Стартовая, дом 27к2,...",2020-05-08 11:28:00+00:00,13990.00,1002
2,9280440300641943,184661359,9492,"ООО ""ПРОФИ""","77 - Г.Москва, 129345, Малыгина ул, 7",2020-06-17 13:37:00+00:00,650.00,1003
3,9287440300660122,2470626943,29195,"ООО ""Аксилог""","127322, Москва, Огородный пр., 20, стр. 23",2020-07-04 12:13:00+00:00,2820.00,1004
4,9251440300055006,3983976056,15343,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МВМ""",Нет данных,2020-08-30 10:56:00+00:00,1290.00,1005
...,...,...,...,...,...,...,...,...
947,9960440502983800,1408934101,44536,"АО ""ТД ""Перекресток""""","129345,77,г.Москва, муниципальный округ Лосино...",2023-06-02 12:43:00+00:00,659.48,1948
948,7282440500223376,864509374,15594,"ООО ""Агроторг""","129336, 77 - ГФЗ Москва, вн.тер.г. муниципаль...",2023-06-03 18:47:00+00:00,635.73,1949
949,7282440500217280,1130390202,1736,"ООО ""Агроторг""","129336, 77 - ГФЗ Москва, вн.тер.г. муниципаль...",2023-06-04 13:07:00+00:00,1397.00,1950
950,7284440500167859,3070544374,10534,"ООО ""Камелот-А""","129336, г.Москва, ул.Стартовая, д.23, к.1",2023-06-04 16:47:00+00:00,36.98,1951


In [6]:
'2018-01-25T18:26:00.000Z'
'2023-02-24T19:51:00.000Z'

'2023-02-24T19:51:00.000Z'

# Purchase DF
- BASE: purches_df
- RAW_DF: only_purch
- purch_columns = ['product','category','quantity','price','cost','_index_purch']

In [14]:
purch_index = base_index

purch_df_raw = df.copy()
only_purch = purch_df_raw \
    .drop(index= store_df.index) \
    .drop(index=store_info_index)

for index, row in only_purch.iterrows():
    if row['c0'] == 'Товар':
        purch_index += 1
        row['c5'] = purch_index
    else:
        row['c5'] = purch_index
purch_columns = ['product','category','quantity','price','cost','_index_purch']        
purch_df_no_columns = only_purch. \
    drop('c6',axis=1). \
    dropna(axis=0,thresh=3) \
    .query("c0 !='Товар'")

purch_df_no_columns.columns  = purch_columns
purches_df = purch_df_no_columns
purches_df[['quantity','cost']] = purches_df[['quantity','cost']].apply(pd.to_numeric)
purches_df['price'] = (purches_df.cost / purches_df.quantity).apply(pd.to_numeric).round(2)



purches_df.loc[purches_df['price'].sort_values(ascending=False).head(50).index]
# purches_df.loc[purches_df.duplicated()]
purches_df.drop_duplicates(inplace=True)
purches_df

,product,category,quantity,price,cost,_index_purch
4,*8041 Нектар ДОБРЫЙ МУЛЬТИФРУТ 2л,Напитки,1.000,96.90,96.90,1001
5,3300573 Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.000,5.00,5.00,1001
6,3404094 СЕЛЯН.Хлеб ДАРНИЦ.пол.320г,Хлеб,1.000,12.90,12.90,1001
7,3182856 Яйца кур.С1 стол.фас.10шт,Мясо и птица,1.000,42.50,42.50,1001
8,"*2085981 ПРОСТ.Мол.отб.3,4-4,5% 930мл",Молочные продукты,1.000,44.90,44.90,1001
...,...,...,...,...,...,...
9869,Томаты,Овощи и фрукты,0.864,94.99,82.07,1953
9870,Крендельки Krutzel Бретцель с солью 250г,Снеки,1.000,39.99,39.99,1953
9871,Лепешки Mission grill Тортильи пш.ориг250г,Хлеб,2.000,79.98,159.96,1953
9872,Дирол XXL подуш.colors фр.ассорти 19г,Чай и сладкое,1.000,29.89,29.89,1953


In [24]:
df_all = purches_df.merge(new_store_df)

# df_all.query("product == 'Кофе в зернах с ароматом Ginger cookies, ароматизированный натуральный кофе Имбирный пряник, 200 гр.'")

# Processing
Derivative

In [22]:
new_store_df.storage_name.value_counts()
purches_df['category'].value_counts()


df_purch_store = new_store_df[['storage_name','adres','_date','sum_rub','_index_purch']] \
    .merge(purches_df, how='inner')
    

some_stat = df_purch_store \
    .groupby('product',as_index=False) \
    .agg(
        agg_price=('price',"mean"),
        quantity=('quantity','sum'),
        agg_cost_cat=('cost',"sum"))\
    .round(0) \
    .sort_values(by='agg_cost_cat',ascending=False) 

general_products = some_stat.query(
    "agg_price < 1500 and agg_price >0 and product != 'ПРИХОД' and product !='Получение аванса'") 
    
general_products.agg_cost_cat.sum() / ((2023 -2018)*12+1)

general_products \
    [general_products['product'].str.contains('Яйц')]



general_products.sort_values(by='agg_cost_cat',ascending=False).head(20)



,product,agg_price,quantity,agg_cost_cat
1249,Яйцо куриное СО (10шт),89.0,216.0,19263.0
765,"Молоко СТМ 2,5% полипак 900г",52.0,183.0,9354.0
1097,Творог Искренне Ваш обезжир 500г пакет,172.0,43.0,7408.0
1102,Творог рассыпчатый Искренне Ваш 5% 500г пакет,170.0,41.0,6970.0
488,Грибы Вешенка 300г,101.0,67.0,6909.0
1247,Яйцо куриное С1 (10шт),68.0,96.0,6669.0
999,Семечки Бабкины соленые 300г,89.0,66.0,5834.0
1241,Яблоки свежий урожай,67.0,86.0,5690.0
1017,Смесь семян для салата Классическая 60г,44.0,124.0,5458.0
998,Семечки Бабкины жареные 300г,98.0,48.0,4661.0


In [8]:
general_products[general_products['product'].str.contains('тво')]

,product,how_much,agg_price,quantity,agg_cost_cat
522,"Десерт Чудо творог 4,2% Черника 100г",4,35.0,4.0,140.0
293,HOCHL.Сыр С ЗЕЛЕН.твор.60% 140г,1,138.0,1.0,138.0
1044,Средство SYNERGETIC д/посуды 500млАпельс,1,104.0,1.0,104.0
1041,Сочник с творогом 110г,1,37.0,1.0,37.0


In [218]:
purches_df.category.value_counts()
purches_df.query('category == "Алкоголь" | category == "Табак"')

,product,category,quantity,price,cost,_index_purch
2160,"Пиво Факс светлое 0,45л ж/б 4,9%",Алкоголь,3.0,56.99,170.97,1185
2327,*СИГАРЕТЫ LD AUT.100'S RED ПАЧКА,Табак,1.0,145.00,145.00,1203
2394,*СИГАРЕТЫ DAVIDOFF REACH BLUE,Табак,1.0,150.00,150.00,1207
2406,Нап.пив.АМС.НЛ.ап/гр.б/а.0.43л,Алкоголь,2.0,46.99,93.98,1209
2407,*Нап.SANTO STEFANO кр.п/сл.0.75л,Алкоголь,1.0,239.99,239.99,1209
...,...,...,...,...,...,...
9164,"Нап пив HOEGAARDEN бел 4,9% ж/б 0.45L",Алкоголь,1.0,85.99,85.99,1877
9201,Коньяк FATHER'S OLD BARREL 5лет 0.25L,Алкоголь,1.0,269.99,269.99,1882
9250,"Вино GALO VALENTE бел. сух8,5-11,5% 1L",Алкоголь,1.0,449.99,449.99,1887
9251,"Пиво EBOSHI СветФильтрПаст4,9%ж/б 0.5L",Алкоголь,3.0,99.89,299.67,1887


In [18]:
df_cofe = general_products[general_products['product'].str.contains('кофе|Кофе')]
# df_cofe

In [31]:
pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 700)
df_cofe.groupby('product',as_index=False).agg(counts=('quantity','sum'),summa =('agg_cost_cat','sum')) \
    .sort_values(by='counts',ascending=False)


,product,category,quantity,price,cost,_index_purch
2208,"Кофе в зернах с ароматом Ginger cookies, ароматизированный натуральный кофе Имбирный пряник, 200 гр.",Чай и сладкое,1.0,165.0,165.0,1191
5858,"Кофе в зернах с ароматом Ginger cookies, ароматизированный натуральный кофе Имбирный пряник, 200 гр.",Чай и сладкое,1.0,165.0,165.0,1532
9436,"Кофе в зернах с ароматом Ginger cookies, ароматизированный натуральный кофе Имбирный пряник, 200 гр.",Чай и сладкое,1.0,165.0,165.0,1907


purches_df.category.values_count()

In [145]:
import seaborn as sns

In [217]:
egg_index = df_purch_store['product'].str.contains('Яйц').index
eggs = df_purch_store \
    .loc[egg_index] \
    .groupby(['price',"_date"]) \
    .agg({"price":'mean'}) \
        .query('price>0')
    # [['price',"_date"]] \
    # .sort_values(by="_date")
eggs
# sns.histplot(data= eggs, x="_date",y="price",bins=10,kde=True)  

price
price    _date                              
0.34     2023-03-04 20:32:00+00:00      0.34
1.00     2022-11-20 14:27:00+00:00      1.00
         2022-11-20 19:00:00+00:00      1.00
         2023-01-10 15:12:00+00:00      1.00
         2023-01-10 15:17:00+00:00      1.00
...                                      ...
15934.00 2023-05-08 16:59:00+00:00  15934.00
         2023-05-10 17:28:00+00:00  15934.00
16399.00 2021-10-04 21:30:00+00:00  16399.00
         2021-11-10 18:56:00+00:00  16399.00
38490.00 2022-12-26 17:01:00+00:00  38490.00

[4023 rows x 1 columns]

In [90]:
import seaborn as sns
# Получение аванса avance
avance = purches_df[purches_df['product'] == 'Получение аванса'] \
    .merge(new_store_df) \
    [['product','category','cost','storage_name','adres']]
# ПРИХОД	
income = purches_df[purches_df['product'] == 'ПРИХОД'] \
    .merge(new_store_df) \
    [['product','category','cost','storage_name','adres']]     
# income

In [89]:
# avance

In [249]:

pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 500)

new_store_df \
    .groupby(['storage_name','adres'],as_index=False) \
    .agg(store_trafic = ('FN','count')) \
    .query('store_trafic >10') \
    .sort_values(by='store_trafic',ascending=False)

,storage_name,adres,store_trafic
73,"ООО ""Камелот-А""","129336, г.Москва, ул.Стартовая, д.23, к.1",316
62,"ООО ""Агроторг""","129336,,,, 77 - Г.Москва, вн.тер.г. муниципальный округ Лосиноостровский, ул Стартовая, Дом 33А,",88
23,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТЕРНЕТ РЕШЕНИЯ""","109316, Москва, Волгоградский проспект, 42, к 9",57
25,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТЕРНЕТ РЕШЕНИЯ""","Москва, Волгоградский проспект, 42, кор. 9",34
63,"ООО ""Агроторг""","129336,,,, 77 - Город федерального значения Москва, вн.тер.г. муниципальный округ Лосиноостровский, ул Стартовая, Дом 33А,",34
61,"ООО ""Агроторг""","129336,,,, 77 - Г.Москва, вн.тер.г. муниципальный округ Лосиноостровский, ул Стартовая, Дом 33А,",32
64,"ООО ""Агроторг""","129336,77, г. Москва, Стартовая ул., 33А",28
72,"ООО ""Камелот-А""","129336, г.Москва, ул.Стартовая, д.23, к.1",26
43,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПОРТАЛ""",Нет данных,21
10,ИП Храмов В.Н.,Нет данных,20


# Process


In [65]:
# 1371
prod = 'Пакет-майка ПНД 29х54 20мкм "Акции ЯРЧЕ"'
print(df_all.shape[0])
df_all['_day'] = df_all._date.dt.date
df_no_dublicates = df_all[~df_all.duplicated(
    subset=['product','category','quantity','price','_day'])]

df_no_dublicates[['product','category','quantity','price','_day']].query('product == @prod') \
    .groupby(['_day','product']).agg( days_count=('_day','count')) \
    .query('days_count >1').shape[0]

4807


1

In [87]:
df_catergoryc = df_no_dublicates.copy()
df_catergoryc['category'] = df_no_dublicates['category'].astype("category")
categoties = df_catergoryc.category.value_counts()
top_cat = categoties.drop('Упаковка').head(10).index

top_cat

CategoricalIndex(['Овощи и фрукты', 'Чай и сладкое', 'Молочные продукты',
                  'Бакалея', 'Снеки', 'Для дома', 'Мясо и птица', 'Хлеб',
                  'Гастрономия', 'Напитки'],
                 categories=['Авто', 'Алкоголь', 'Бакалея', 'Гастрономия', ..., 'Услуги', 'Хлеб', 'Чай и сладкое', 'Электроника'], ordered=False, dtype='category')

In [273]:
df_products_only_common = (df_catergoryc
    [['product'	,'category',	'quantity',	'price','_date',	'sum_rub',	'_day']]
    .query("category not in ['Услуги','Упаковка','Для дома','Электроника']")
    .query("price < 1_400 ")
    .query("product not in ['Получение аванса']")
    )

    
(df_products_only_common
    .groupby('product',as_index=False).agg(
        price_mean= ('price','mean'),
        day_count = ('_day',pd.Series.nunique),
        day_min =   ('_date',pd.Series.min),
        day_max =   ('_date',pd.Series.max))
    .query('day_count >15')
    .sort_values(by='day_min',ascending=True)
    )

,product,price_mean,day_count,day_min,day_max
1093,Яйцо куриное СО (10шт),89.364773,86,2020-05-08 11:28:00+00:00,2023-05-31 21:51:00+00:00
327,АлпенГолд темный шок с дроб. фундуком 80г,54.990000,16,2020-05-08 11:28:00+00:00,2023-04-09 11:12:00+00:00
962,Творог рассыпчатый Искренне Ваш 5% 500г пакет,169.990000,33,2020-05-08 11:28:00+00:00,2023-05-02 17:53:00+00:00
874,Семечки Бабкины соленые 300г,89.948800,50,2020-05-08 11:28:00+00:00,2023-06-02 12:43:00+00:00
680,"Молоко СТМ 2,5% полипак 900г",51.646571,35,2020-05-08 11:28:00+00:00,2023-05-31 21:51:00+00:00
1085,Яблоки свежий урожай,66.564375,54,2021-05-26 21:11:00+00:00,2023-03-25 21:12:00+00:00
969,Томаты,104.294444,32,2021-05-26 21:11:00+00:00,2023-05-31 21:51:00+00:00
679,"Молоко СТМ 2,5% ПЭТ 950г",48.480294,30,2021-05-26 21:11:00+00:00,2023-02-23 21:02:00+00:00
751,Огурцы гладкие среднеплодн.,82.725789,18,2021-05-26 21:11:00+00:00,2023-03-06 20:42:00+00:00
1046,Шампиньоны 300г,80.942609,23,2021-05-28 13:45:00+00:00,2023-04-17 16:02:00+00:00


In [218]:
df_top_products_by_day = df_catergoryc \
    .groupby(['product'],as_index=False).agg(count_items = ('_day',pd.Series.nunique)) \
    .sort_values(by='count_items',ascending=False) \
    
df_top_products_by_day

,product,count_items
882,"Пакет-майка ПНД 40х64 30мкм ""Акции ЯРЧЕ""",144
765,"Молоко СТМ 2,5% полипак 900г",107
1249,Яйцо куриное СО (10шт),86
875,Пакет ПЯТЕРОЧКА 65х40см,81
1241,Яблоки свежий урожай,54
1017,Смесь семян для салата Классическая 60г,52
999,Семечки Бабкины соленые 300г,50
488,Грибы Вешенка 300г,44
881,"Пакет-майка ПНД 29х54 20мкм ""Акции ЯРЧЕ""",43
980,Салат в горшочке,39


In [211]:
import seaborn as sns
import matplotlib.pyplot as plt
time_series_df = df_catergoryc.query('product == "Молоко СТМ 2,5% полипак 900г"') \
    .groupby('_date',as_index=False)[['price']].agg('mean')
# 'quantity',
# sns.jointplot(data=time_series_df, x='_day',y='price',kind='hist')
# sns.pairplot(data=time_series_df,hue='quantity')

# sns.displot(data=time_series_df,x='_date',kde=True,alpha=0.6,kind='hist',multiple="stack")
def difference(product_name):
    time_series_df = df_catergoryc[df_catergoryc['product'] == product_name] \
    .groupby('_date',as_index=False)[['price']].agg('mean')

    mi,ma = time_series_df[['price']].min(),time_series_df[['price']].max()
    return (ma-mi)/mi
    
# time_series_df
df_top_products_by_day['diff'] = df_top_products_by_day['product'].apply(difference)

difference("Молоко СТМ 2,5% полипак 900г")
# df_catergoryc[df_catergoryc['product'] == "Молоко СТМ 2,5% полипак 900г"]
df_top_products_by_day.sort_values(by='diff',ascending=False)

,product,count_items,diff
914,Получение аванса,34,193.911315
950,Роялти,13,77.727273
875,Пакет ПЯТЕРОЧКА 65х40см,81,23.970588
253,COOL COLA Нап.ZERO б/а.ПЭТ 1.5л,15,4.220632
853,Орех грецкий очищенный 1кг,14,3.299432
773,Морковь вес,14,2.212143
841,Огурцы гладкие среднеплодн.,18,1.889951
1111,Томаты 1кг,27,1.819170
1110,Томаты,32,1.500375
602,Капуста Пекин,21,1.111235


In [215]:
df_catergoryc.query('product == "Пакет ПЯТЕРОЧКА 65х40см"')
# df_catergoryc.loc[3862]['price'] = 123.75

,product,category,quantity,price,cost,_index_purch,FN,OPD,FD,storage_name,adres,_date,sum_rub,_day
45,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,5.99,5.99,1010,9280440301231563,2646761765,30786,"ООО ""Агроторг""","129336,77, г. Москва, Стартовая ул., 33А",2021-05-29 20:00:00+00:00,5.99,2021-05-29
49,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,5.99,5.99,1012,9280440301234149,328986461,49789,"ООО ""Агроторг""","129336,77, г. Москва, Стартовая ул., 33А",2021-06-01 21:20:00+00:00,830.88,2021-06-01
79,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,5.99,5.99,1015,9280440301231496,2875277503,43426,"ООО ""Агроторг""","129336,77, г. Москва, Стартовая ул., 33А",2021-06-05 18:20:00+00:00,774.87,2021-06-05
165,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,5.99,5.99,1023,9280440301234149,1976403768,61725,"ООО ""Агроторг""","129336,77, г. Москва, Стартовая ул., 33А",2021-06-29 19:38:00+00:00,510.13,2021-06-29
183,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,5.99,5.99,1026,9280440301231563,3285540543,37373,"ООО ""Агроторг""","129336,77, г. Москва, Стартовая ул., 33А",2021-07-12 12:53:00+00:00,932.92,2021-07-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,8.49,8.49,1766,9960440301295595,2528884949,185474,"ООО ""Камелот-А""","129336, г.Москва, ул.Стартовая, д.23, к.1",2023-03-25 21:07:00+00:00,983.51,2023-03-25
4081,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,8.49,8.49,1771,7282440500068948,1102638673,2437,"ООО ""Агроторг""","129336,,,, 77 - Город федерального значения Мо...",2023-03-27 19:13:00+00:00,405.67,2023-03-27
4320,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,8.49,8.49,1831,7282440500068926,453658434,17908,"ООО ""Агроторг""","129345,77,ГФЗ Москва, вн.тер.г. муниципальный ...",2023-04-22 17:05:00+00:00,88.98,2023-04-22
4343,Пакет ПЯТЕРОЧКА 65х40см,Упаковка,1.0,8.49,8.49,1837,9960440301295595,2602974143,200132,"ООО ""Камелот-А""","129336, г.Москва, ул.Стартовая, д.23, к.1",2023-04-25 20:07:00+00:00,946.84,2023-04-25


# deep procsessing
## 

In [235]:
df_catergoryc
df_top_products_by_day.head()

,product,count_items
882,"Пакет-майка ПНД 40х64 30мкм ""Акции ЯРЧЕ""",144
765,"Молоко СТМ 2,5% полипак 900г",107
1249,Яйцо куриное СО (10шт),86
875,Пакет ПЯТЕРОЧКА 65х40см,81
1241,Яблоки свежий урожай,54
